# Compiling all options

In [ ]:
from mqt.predictor import utils, ml
from mqt.bench import get_benchmark
from pathlib import Path

import numpy as np
import time

pred = ml.Predictor()
qc = get_benchmark("dj", 1, 7)
source_path = Path("runtime_comp")
target_path = Path("runtime_comp_compiled")

if not source_path.exists():
    source_path.mkdir()
if not target_path.exists():
    target_path.mkdir()
    
    
filename = "dj_indep_qiskit_7.qasm"
qc.qasm(filename=str(source_path/filename))

results = []
for _ in range(5):
    start = time.time()
    pred.compile_all_circuits_for_qc(
        filename=filename, source_path=str(source_path), target_path=str(target_path), timeout=60
    )
    res = pred.generate_training_sample(file=filename.split(".")[0], path_uncompiled_circuit=str(source_path), path_compiled_circuits=str(target_path))
    end = time.time()
    diff_all_compile = end - start
    results.append(diff_all_compile)
runtimes_all_comp = results

# Using MQT Predictor

In [ ]:
results = []
for _ in range(5):
    start = time.time()
    prediction = pred.predict(str(source_path/filename))
    print(prediction)
    pred.compile_predicted_compilation_path(str(source_path/filename), prediction)
    end = time.time()

    diff_predictor = round((end - start), 3)
    results.append(diff_predictor)

runtimes_predictor = results

# Difference

In [ ]:
runtimes_all_comp

In [ ]:
runtimes_predictor

In [ ]:
diff = 1 - np.mean(runtimes_predictor) / np.mean(runtimes_all_comp)

In [ ]:
print("Runtime all compilations: ", np.mean(runtimes_all_comp))
print("Runtime Predictor: ", np.mean(runtimes_predictor))
print("Percentual Difference: ", np.round(diff, 3))

# Check many samples randomly

In [ ]:
from mqt.predictor import utils, ml
from mqt.bench import get_benchmark
from pathlib import Path

import numpy as np
import time



pred = ml.Predictor()

source_path = ml.helper.get_path_training_circuits()
target_path = Path("runtime_comp_compiled")

#if not source_path.exists():
#    source_path.mkdir()
if not target_path.exists():
    target_path.mkdir()
    
    
results = []

file_list = list(ml.helper.get_path_training_circuits().glob("*.qasm"))

compile_time_all = []
compile_time_pred = []
for i in range(20):
    random_index = np.random.randint(len(file_list))
    filename = file_list[random_index]
    print(i, filename)
    
    start = time.time()
    pred.compile_all_circuits_for_qc(filename=str(filename.name), source_path=str(source_path), target_path=str(target_path), timeout=60000)
    res = pred.generate_training_sample(file=str(filename.stem), path_compiled_circuits=str(target_path))
    end = time.time()
    compile_time_all.append(end - start)
    
    
    start = time.time()
    prediction = pred.predict(str(source_path/filename))
    pred.compile_predicted_compilation_path(str(source_path/filename), prediction)
    end = time.time()
    compile_time_pred.append(end - start)
runtimes_all_comp = results

In [ ]:
compile_time_all

In [ ]:
compile_time_pred

In [ ]:
rel = np.array(compile_time_all)/np.array(compile_time_pred)
rel

In [ ]:
print(np.median(rel), np.mean(rel))